In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
import pandas as pd
import h5py

import sys
sys.path.append("../../efish-physics-model/objects")
sys.path.append("../../efish-physics-model/helper_functions")
sys.path.append("../../efish-physics-model/uniform_points_generation")

In [11]:
# h5py_file = h5py.File(f"/home/ssd2tb/dturcu/electric_fish_processed_data/data-2024_06_13-characterization_dataset/responses.hdf5",'r')["responses"]
h5py_file_chunked_manual = h5py.File(f"/home/ssd2tb/dturcu/electric_fish_processed_data/data-2024_06_13-characterization_dataset/responses-chunked.hdf5",'r')["test-chunking"]
h5py_file_chunked_manual_main = h5py.File(f"hdf5-chunking/responses-chunked.hdf5",'r')["test-chunking"]


In [ ]:
### manually chunk the dataset
# with h5py.File(f"hdf5-chunking/responses-chunked.hdf5", 'w') as f:
    # f.create_dataset("test-chunking", shape=h5py_file.shape, dtype=h5py_file.dtype, chunks=tuple([1]+list(h5py_file.shape[1:])))
### auto-chunk the dataset
with h5py.File(f"hdf5-chunking/responses-chunked-auto.hdf5", 'w') as f:
    f.create_dataset("test-chunking-auto", shape=h5py_file.shape, dtype=h5py_file.dtype, chunks=True)

In [ ]:
with h5py.File(f"hdf5-chunking/responses-chunked-auto.hdf5", "r+") as f:
    write_size = 100_000
    for i in range(0, h5py_file.shape[0], write_size):
        print(i//write_size, end=", ")
        f["test-chunking-auto"][i : i + write_size] = h5py_file[i : i + write_size]
        if (i/write_size+1) % 50 == 0:
            print()

In [ ]:
# h5py_file_chunked_manual = h5py.File(f"hdf5-chunking/responses-chunked.hdf5",'r')["test-chunking"]
# h5py_file_chunked_auto = h5py.File(f"hdf5-chunking/responses-chunked-auto.hdf5",'r')["test-chunking-auto"]

In [ ]:
ids = np.random.permutation(h5py_file_chunked_manual.shape[0])[:3]
ids = np.sort(ids)
(ids, #h5py_file[ids], h5py_file_chunked_manual[ids], h5py_file_chunked_auto[ids], 
(h5py_file[ids] == h5py_file_chunked_manual[ids]).all(), 
# (h5py_file[ids] == h5py_file_chunked_auto[ids]).all()
)

In [15]:
ids = np.random.permutation(h5py_file_chunked_manual.shape[0])[:27000]
# ids = np.sort(ids)
t0 = time.time()
for i in ids:
    _ = h5py_file_chunked_manual_main[i]
t1 = time.time()
for i in ids:
    _ = h5py_file_chunked_manual[i]
t2 = time.time()
print(f"Time for non-chunked: {t1-t0:.2f} s")
print(f"Time for man chunked: {t2-t1:.2f} s")
# for i in ids:
#     _ = h5py_file_chunked_auto[i]
# t3 = time.time()
# print(f"Time for aut chunked: {t3-t2:.2f} s")

Time for non-chunked: 7.96 s
Time for man chunked: 6.53 s
